In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import VectorAssembler
import pickle
import pandas as pd

In [2]:
conf = SparkConf() \
    .setAppName("linreg_pleio_vs_AF")\

# Create a SparkContext with the specified configurations
if 'spark' in locals() and spark!=None:
    spark.stop()

sc = SparkContext(conf=conf)

# Create a SparkSession from the SparkContext
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/15 18:28:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/15 18:28:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/15 18:28:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/02/15 18:28:17 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/02/15 18:28:17 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/02/15 18:28:17 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [3]:
#loading in all autosomes
#Skipping sex chromosomes, see readme
df = spark.read \
    .option("comment", "#") \
    .option("delimiter", ",") \
    .csv("/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_data/2.filter/*.csv/*.csv.gz", header=True)

24/02/15 18:28:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df

DataFrame[CHROM: string, POS: string, ID: string, REF: string, ALT: string, QUAL: string, FILTER: string, INFO: string, K562__ref: string, HepG2__ref: string, SKNSH__ref: string, K562__alt: string, HepG2__alt: string, SKNSH__alt: string, K562__skew: string, HepG2__skew: string, SKNSH__skew: string, AC: string, AN: string, AF: string, cadd_phred: string, is_in_dELS: string, is_in_CA: string, is_in_pELS: string, is_in_CA-H3K4me3: string, is_in_CA-CTCF: string, is_in_PLS: string, is_in_TF: string, is_in_CA-TF: string, P_ANNO: string, mean_ref: string, mean_skew: string, MAF: string, category: string]

In [5]:
df_cre=df

In [6]:
cell_types=["K562","SKNSH","HepG2"]
for cell_type in cell_types:
    df_cre = df_cre.withColumn(f"emVar_{cell_type}", 
                           (F.abs(F.col(f"{cell_type}__skew")) >= 0.5) & 
                           (F.greatest(F.col(f"{cell_type}__ref"), F.col(f"{cell_type}__alt")) >= 1.0))

#next, we count the number of cell-types each variant is an emvar in to compute the pleitropy. 
df_cre = df_cre.withColumn("pleio", F.col("emVar_K562").cast("int") + F.col("emVar_SKNSH").cast("int") + F.col("emVar_HepG2").cast("int"))

In [7]:
df_cre=df_cre.withColumn("pleio",F.col("pleio").cast("int"))
df_cre=df_cre.withColumn("AF",F.col("AF").cast("float"))

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df_cre.where(F.col("pleio")>0).limit(10).toPandas())


24/02/15 18:28:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,K562__ref,HepG2__ref,SKNSH__ref,K562__alt,HepG2__alt,SKNSH__alt,K562__skew,HepG2__skew,SKNSH__skew,AC,AN,AF,cadd_phred,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,is_in_PLS,is_in_TF,is_in_CA-TF,P_ANNO,mean_ref,mean_skew,MAF,category,emVar_K562,emVar_SKNSH,emVar_HepG2,pleio
0,chr13,18275436,.,G,C,.,PASS,K562__ref=0.7542868;HepG2__ref=1.0054506;SKNSH...,0.7542868,1.0054506,0.2766882,0.87993294,1.5404036,0.32129806,0.12564613,0.53495294,0.044609863,1,149040,0.000007,4.274,true,false,false,false,false,false,false,false,-0.145,0.6788085301717123,0.2350696325302124,6.70961E-6,SINGLETON,False,False,True,1
1,chr13,18423237,rs1462453604,C,A,.,PASS,K562__ref=2.119739;HepG2__ref=1.989214;SKNSH__...,2.119739,1.989214,1.8035933,0.59563404,0.5352767,0.105299234,-1.5241051,-1.4539372,-1.698294,8064,148982,0.054127,3.425,false,false,false,false,false,false,false,false,0.338,1.9708487192789714,-1.5587787628173828,0.0541273,COMMON,True,True,True,3
2,chr13,18432376,.,G,T,.,PASS,K562__ref=1.459229;HepG2__ref=1.3713883;SKNSH_...,1.459229,1.3713883,1.9209701,0.7357115,0.5304932,0.816227,-0.72351754,-0.84089524,-1.1047434,1,148684,0.000007,0.345,false,false,false,false,false,false,false,false,0.632,1.5838624636332195,-0.8897186915079752,6.72567E-6,SINGLETON,True,True,True,3
3,chr13,18432376,rs1423281620,G,A,.,PASS,K562__ref=1.459229;HepG2__ref=1.3713883;SKNSH_...,1.459229,1.3713883,1.9209701,0.7177803,0.510151,0.7950688,-0.7414487,-0.8612374,-1.1259016,5,148686,0.000034,0.44,false,false,false,false,false,false,false,false,0.632,1.5838624636332195,-0.9095292091369629,3.36279E-5,ULTRARARE,True,True,True,3
4,chr13,18549663,rs1374572071,A,G,.,PASS,K562__ref=1.1435362;HepG2__ref=0.3178485;SKNSH...,1.1435362,0.3178485,0.27461162,2.2180665,0.86089796,1.0274059,1.0745302,0.5430495,0.75279415,1,141638,0.000007,2.84,false,false,false,false,false,false,false,false,0.075,0.5786654551823934,0.7901245752970377,7.06025E-6,SINGLETON,True,True,False,2
5,chr13,18568178,rs1323799600,C,T,.,PASS,K562__ref=2.5305932;HepG2__ref=1.9985994;SKNSH...,2.5305932,1.9985994,1.734408,0.8109198,1.0498794,1.1158385,-1.7196734,-0.94872,-0.6185695,8,152146,0.000053,4.355,false,false,false,true,false,false,false,false,0.155,2.08786678314209,-1.0956543286641438,5.25811E-5,ULTRARARE,True,True,True,3
6,chr13,18612611,rs1250099200,C,G,.,PASS,K562__ref=0.48280463;HepG2__ref=0.5020316;SKNS...,0.48280463,0.5020316,0.5023075,1.0121182,0.8874643,0.74475884,0.52931345,0.38543266,0.24245133,1,152212,0.000007,11.31,false,false,false,false,false,false,false,false,0.252,0.49571458498636883,0.3857324918111165,6.56978E-6,SINGLETON,True,False,False,1
7,chr13,18623488,rs1472182424,T,C,.,PASS,K562__ref=0.46271077;HepG2__ref=0.3666604;SKNS...,0.46271077,0.3666604,0.446558,0.8189979,0.81054384,1.1872326,0.35628715,0.44388345,0.74067456,1,151990,0.000007,5.397,false,false,false,false,false,false,false,false,-2.259,0.4253097375233968,0.5136150519053141,6.57938E-6,SINGLETON,False,True,False,1
8,chr13,18639264,rs557726455,A,C,.,PASS,K562__ref=2.382477;HepG2__ref=1.6495326;SKNSH_...,2.382477,1.6495326,2.0630422,1.2952616,0.84407073,1.2353448,-1.0872154,-0.80546176,-0.82769734,1,151186,0.000007,3.926,false,false,false,false,false,false,false,false,0.976,2.031683921813965,-0.9067915280659994,6.61437E-6,SINGLETON,True,True,True,3
9,chr13,18639264,rs557726455,A,T,.,PASS,K562__ref=2.382477;HepG2__ref=1.6495326;SKNSH_...,2.382477,1.6495326,2.0630422,0.83678895,0.5275409,0.8193733,-1.5456883,-1.1219918,-1.2436689,25,151186,0.000165,3.768,false,false,false,false,false,false,false,false,0.976,2.031683921813965,-1.3037829399108887,1.65359E-4,RARE,True,True,True,3


In [9]:
df_cre = df_cre.filter(~(F.isnan("pleio") | F.col("pleio").isNull() | F.isnan("AF") | F.col("AF").isNull()))

In [10]:
df_intoml=df_cre["pleio","AF"]

In [11]:
assembler = VectorAssembler(inputCols=["pleio"], outputCol="features")
df_intoml = assembler.transform(df_cre).select("features","AF")

In [12]:
from pyspark.ml.regression import LinearRegression

In [13]:
lr = LinearRegression(featuresCol="features", labelCol="AF")
lr_model = lr.fit(df_intoml)

24/02/15 18:28:39 WARN Instrumentation: [9c7c8450] regParam is zero, which might cause numerical instability and overfitting.
24/02/15 18:28:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/02/15 18:28:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/02/15 18:40:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
lr_model.write().overwrite().save("./pleio_vs_af_model")

In [15]:
genome_regions=["is_in_PLS","is_in_dELS","is_in_pELS"]

In [ ]:
for var in genome_regions:
    df_intoml=df_cre.where(F.col(var)==True)
    df_intoml=df_intoml["pleio","AF"]
    assembler = VectorAssembler(inputCols=["pleio"], outputCol="features")
    df_intoml = assembler.transform(df_cre).select("features","AF")
    lr = LinearRegression(featuresCol="features", labelCol="AF")
    lr_model = lr.fit(df_intoml)
    lr_model.write().overwrite().save("./pleio_vs_af_model_"+var)
    

24/02/15 18:53:06 WARN Instrumentation: [acb51c9c] regParam is zero, which might cause numerical instability and overfitting.
24/02/15 19:25:48 WARN Instrumentation: [87b30b88] regParam is zero, which might cause numerical instability and overfitting.


Now let us examine some interesting statistics...

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

In [ ]:
for i in ["pleio_vs_af_model"]+["pleio_vs_af_model_"+x for x in genome_regions]:
    lr_model = LinearRegressionModel.load(i)
    b=lr_model.intercept
    slope=lr_model.coefficients
    r_squared="err"
    try:
        r_squared = lr_model.summary.r2
    except:
        pass
    print(f"{i} {b} {slope} {r_squared}")
    